In [6]:
import pandas as pd
from discovery_child_development.utils import taxonomy_labelling_utils as tlu

In [36]:
df = pd.read_parquet("taxonomy_labelled_data_test.parquet")
df.head(3)

,id,text,source,cost,labels_raw,labels
237,CN-212523035-U,Splicing windmill. The utility model belongs t...,patents,0.001416,[Expressive arts and design],[Expressive arts and design]
322,CN-217146258-U,A children&#39;s bicycle with support assist f...,patents,0.001420,[Physical],[Physical]
978,W4388106491,Nurturing Healthy Media Consumption Habits in ...,openalex,0.001447,"[Communication and language, Media, Assessment...","[Communication and language, Media, Assessment..."


In [7]:
categories_flat = tlu.load_categories()

In [8]:
categories_flat

{'Cognitive development': 'cognition, memory, problem-solving, executive function',
 'SEND': 'special needs, disabilities, learning difficulties, autism, dyslexia, attention-deficit hyperactivity disorder',
 'Communication and language': 'language acquisition, language development, communication skills, speech',
 'Physical': 'motor skills, physical development, spatial awareness',
 'Expressive arts and design': 'creativity, arts, design, imagination, singing, music',
 'Prenatal': 'prenatal development, maternal health, fetal development, pregnancy, birth',
 'Literacy': 'reading, writing, letters, handwriting, literacy development, phonics',
 'Infancy': 'age 0-2, infancy, newborns, early infancy',
 'Mathematics': 'math, numbers, numeracy, mathematical skills',
 'Sleep': 'sleep patterns, sleep quality, sleep duration, bedtime routines',
 'Personal social emotional': 'social development, emotional development, self-awareness, relationships',
 'Early childhood development (general)': 'ages

In [9]:
from discovery_child_development.utils.utils import load_jsonl

data = load_jsonl("training_validation_data_patents_openalex.jsonl")

In [12]:
data[0]

{'id': 'W1519846160',
 'text': 'Intricate Identities: Cochlear Implant Users Negotiating Lives Between d/Deaf and Hearing Worlds. &lt;p&gt;The cochlear implant (CI), a device that “provides hearing sensations for severely and profoundly deaf individuals” (NZ Audiological Society), initially emerged for public use in the 1980s, but was met with strong opposition from Deaf communities in many countries (Lane et al 1996, Edwards 2005). However, since the beginning of the 21st century, hostility towards implants has lessened and they are increasingly accepted as an option in a range of possibilities for deaf children and adults. Despite increasing numbers of the Deaf community considering implants themselves, however, the continuing task of the Deaf community is to counter the conception of implants as ‘miracle cures’ for deafness (Lane et al 1996, Edwards 2005). Furthermore, the Deaf community needs to communicate to parents of deaf children that those with implants may still be perceived

In [1]:
import argparse
import json
import pandas as pd
import asyncio
import time
from pathlib import Path

from nesta_ds_utils.loading_saving import S3
from discovery_child_development import logging, PROJECT_DIR, S3_BUCKET
from discovery_child_development.getters import get_dataset
from discovery_child_development.utils.openai_utils import (
    MessageTemplate,
    Classifier,
    FunctionTemplate,
)
from discovery_child_development.utils.labelling_utils import (
    create_category_description_string,
)
from discovery_child_development.utils.utils import (
    load_jsonl,
    create_directory_if_not_exists,
    batch,
    get_yaml_config,
)
import os 

# Get labelling config params
CONFIG = get_yaml_config(Path(os.path.abspath('')) / "config.yaml")
# Define paths to the prompt and function definitions
PATH_TO_PROMPTS = Path(os.path.abspath('')) / "prompts"
PATH_TO_CATEGORIES = PATH_TO_PROMPTS / "categories.json"
PATH_TO_MESSAGE_PROMPT = PATH_TO_PROMPTS / "prompt.json"
PATH_TO_FUNCTION = PATH_TO_PROMPTS / "function.json"
# PATH_TO_EXAMPLES = PATH_TO_PROMPTS / "examples.jsonl"
# Define paths to the outputs
OUTPUT_FILEPATH = PROJECT_DIR / CONFIG["local_output_directory"]

2024-01-05 18:09:41,676 - botocore.credentials - INFO - Found credentials in environment variables.


In [3]:
from discovery_child_development.utils import taxonomy_labelling_utils as tlu

In [4]:
categories_flat = tlu.load_categories(PATH_TO_CATEGORIES)

In [6]:
print(create_category_description_string(
                        categories_flat, randomise=True
                    ))

Technology (general): technology, digital, ICT, edtech, tech
Education: early childhood education, early learning, early education, preschool, pre-primary, kindergarten
Data Science and AI: data science, data analysis, artificial intelligence, machine learning, automated speech recognition, computer vision
Expressive arts and design: creativity, arts, design, imagination, singing, music
AR VR: augmented reality, virtual reality, immersive technology
Inclusion: inclusion, equity, equality, diversity, racism, structural racism, colonialism
Income: family income, socioeconomic status, economic factors, cash transfers
Personal social emotional: social development, emotional development, self-awareness, relationships
Mathematics: math, numbers, numeracy, mathematical skills
Early childhood development (general): ages 0-5, early childhood development, early years, early childhood
Labour market: parental employment, work-life balance, parental leave, gender pay gap
Policy: early childhood edu

In [37]:
function = tlu.format_function(categories_flat, PATH_TO_FUNCTION)

In [41]:
function = json.loads(open(PATH_TO_FUNCTION).read())
function



{'name': 'predict_labels',
 'description': 'Predict category labels to the given text',
 'parameters': {'type': 'object',
  'properties': {'prediction': {'type': 'array',
    'items': {'type': 'string', 'enum': []},
    'description': 'Predicted labels to the given text'}},
  'required': ['prediction']}}

In [42]:
function["parameters"]["properties"]["prediction"]["items"]["enum"] = list(categories_flat.keys())
function = FunctionTemplate.load(function)

In [1]:
from discovery_child_development.getters import get_dataset

df

2024-01-05 16:51:36,669 - botocore.credentials - INFO - Found credentials in environment variables.


In [2]:
df

In [8]:
from discovery_child_development.getters import taxonomy
from discovery_child_development import getters

In [12]:
import importlib
importlib.reload(getters);

In [15]:
df = getters.get_dataset("taxonomy_labelling_sample")

2024-01-05 16:53:56,155 - root - INFO - File data/labels/taxonomy_classifier/training_validation_data_patents_openalex.jsonl downloaded from discovery-iss to /Users/karlis.kanders/Documents/code/discovery_child_development/inputs/data/labelling/taxonomy/input/training_validation_data_patents_openalex.jsonl


In [16]:
df

,id,text,source
0,W1519846160,Intricate Identities: Cochlear Implant Users N...,openalex
1,W1523874198,Early Bilingual Education. In the coming years...,openalex
2,W1535586210,The Use of Music Therapy Strategies to Enhance...,openalex
3,W1538577929,Experience-based language acquisition: a compu...,openalex
4,W1556618260,"A new old age?: Exploring the values, attitude...",openalex
...,...,...,...
8833,CN-215456599-U,Environment-friendly children learn drawer for...,patents
8834,US-2021022682-A1,Advanced play environment for screening and ea...,patents
8835,CN-113254836-B,Intelligent child-care knowledge point informa...,patents
8836,CN-110122881-A,A nutritional composition for promoting height...,patents


In [7]:
import pandas as pd

pd.DataFrame(taxonomy.get_labelling_sample())


2024-01-05 16:52:23,442 - root - INFO - File data/labels/taxonomy_classifier/training_validation_data_patents_openalex.jsonl downloaded from discovery-iss to /Users/karlis.kanders/Documents/code/discovery_child_development/inputs/data/labelling/taxonomy/input/training_validation_data_patents_openalex.jsonl


,id,text,source
0,W1519846160,Intricate Identities: Cochlear Implant Users N...,openalex
1,W1523874198,Early Bilingual Education. In the coming years...,openalex
2,W1535586210,The Use of Music Therapy Strategies to Enhance...,openalex
3,W1538577929,Experience-based language acquisition: a compu...,openalex
4,W1556618260,"A new old age?: Exploring the values, attitude...",openalex
...,...,...,...
8833,CN-215456599-U,Environment-friendly children learn drawer for...,patents
8834,US-2021022682-A1,Advanced play environment for screening and ea...,patents
8835,CN-113254836-B,Intelligent child-care knowledge point informa...,patents
8836,CN-110122881-A,A nutritional composition for promoting height...,patents


In [29]:
from discovery_child_development.utils import taxonomy_labelling_utils as tlu

In [30]:
categories_flat = tlu.load_categories()

In [33]:
def unique_list(series):
    return list(set(series))

In [45]:
category_names = list(categories_flat.keys())
keyword_to_category_dict = tlu.make_keyword_dict(categories_flat)
labelled_data = load_jsonl(str(OUTPUT_FILEPATH) + "/testing.jsonl")

cleaned_data = (
    pd.DataFrame(labelled_data)
    .explode("prediction")
    .rename(columns={"prediction": "prediction_raw"})
    .assign(prediction=lambda df: df["prediction_raw"].apply(lambda x: tlu.map_keywords_to_categories(x, keyword_to_category_dict, category_names)))
    .query("prediction != 'no label'")
    .groupby(["id", "text", "source"])
    .agg({"prediction_raw": unique_list, "prediction": unique_list})
    .reset_index()
)

In [46]:
cleaned_data

,id,text,source,prediction_raw,prediction
0,BR-102021009775-A2,MUSICALIZATION TEACHING SYSTEM. MUSICALIZATION...,patents,"[Technology (general), Expressive arts and des...","[Technology (general), Expressive arts and des..."
1,CN-111304915-A,Preparation method and application of bird&#39...,patents,"[Technology (general), Cognitive development, ...","[Technology (general), Cognitive development, ..."
2,CN-113037311-A,A wristband for indoor positioning and navigat...,patents,[Technology (general)],[Technology (general)]
3,CN-116636716-A,A two-way reciprocating motion mechanism and b...,patents,[Technology (general)],[Technology (general)]
4,CN-212631729-U,A child&#39;s robotic arm grips a toy. The uti...,patents,"[Early childhood development (general), Robotics]","[Early childhood development (general), Robotics]"
5,CN-213406708-U,Shading umbrella for infant radiation table. T...,patents,"[Cognitive development, Health, Physical, Slee...","[Cognitive development, Health, Physical, Slee..."
6,CN-218513079-U,A kind of children&#39;s puzzle card learning ...,patents,"[Technology (general), Cognitive development, ...","[Technology (general), Cognitive development, ..."
7,RU-202478-U1,A device for connecting a pipe and two flat pa...,patents,"[Technology (general), Games]","[Technology (general), Games]"
8,US-10966861-B2,Apparatus and method for maintaining enthalpy ...,patents,"[Infancy, Technology (general), Physical]","[Infancy, Technology (general), Physical]"
9,W3038350132,Contextual Effect of Preschool on the Developm...,openalex,"[Income, Cognitive development, Health, Family...","[Income, Cognitive development, Health, Family..."


In [39]:
labelled_data_cleaned

,id,text,source,prediction_raw,prediction
0,CN-111304915-A,Preparation method and application of bird&#39...,patents,"[Technology (general), Cognitive development, ...","[Technology (general), Cognitive development, ..."
1,CN-113037311-A,A wristband for indoor positioning and navigat...,patents,[Technology (general)],[Technology (general)]
2,CN-212631729-U,A child&#39;s robotic arm grips a toy. The uti...,patents,"[Early childhood development (general), Robotics]","[Early childhood development (general), Robotics]"
3,CN-213406708-U,Shading umbrella for infant radiation table. T...,patents,"[Cognitive development, Health, Physical, Slee...","[Cognitive development, Health, Physical, Slee..."
4,CN-218513079-U,A kind of children&#39;s puzzle card learning ...,patents,"[Technology (general), Cognitive development, ...","[Technology (general), Cognitive development, ..."
5,RU-202478-U1,A device for connecting a pipe and two flat pa...,patents,"[Technology (general), Games]","[Technology (general), Games]"
6,US-10966861-B2,Apparatus and method for maintaining enthalpy ...,patents,"[Infancy, Technology (general), Physical]","[Infancy, Technology (general), Physical]"
7,W3195245604,Association between prenatal and early life ho...,openalex,"[Infancy, Child protection, Health, Prenatal]","[Infancy, Child protection, Health, Prenatal]"
8,W4303879366,Early Childhood Exposures to Fluorides and Chi...,openalex,"[Early childhood development (general), Health...","[Early childhood development (general), Health..."
9,WO-2022015497-A3,Method and system for preemptive planning. A m...,patents,"[Technology (general), Operations, Data, Data ...","[Technology (general), Operations, Data, Data ..."
